# Tidy data

In [ ]:
# import ee
# !pip install geopandas geemap geehydro
# import geemap
# import folium
# import geehydro

# try:
#         ee.Initialize()
# except Exception as e:
#         ee.Authenticate()
#         ee.Initialize()
import os
from google.colab import files

import pandas as pd
import altair as alt
import numpy as np
import io
from google.colab import data_table
data_table.enable_dataframe_formatter()

In [ ]:
uploaded = files.upload()
df = pd.read_csv(io.BytesIO(uploaded['MAIN_S2_secchilakes_stats.csv']))

#remove index and geometry columns
df1 = df.iloc[0:,2: ]

In [ ]:
import datetime
from pandas.tseries.offsets import Day
import numpy as np
test = df1.melt(['STORETID', 'COUNT_STOR', 'MEAN_Latit', 'MEAN_Longi'], var_name = "imageID", value_name = "values")
test = test.dropna()
test[["satellite", "date", "delete_date", "tile", "band"]] = test["imageID"].str.split('_', expand=True)
test = test.drop(['delete_date', 'imageID'], axis=1)
test[["capture_date", "delete"]] = test["date"].str.split('T', expand=True)
test = test.drop(['date', 'delete'], axis=1)
test["capture_date"] = pd.to_datetime(test["capture_date"], format='%Y%m%d')
test["date_max"] = test["capture_date"] + pd.Timedelta(days=7)
test["date_min"] = test["capture_date"] - pd.Timedelta(days=7)

#remove duplicates
# test = test.round(decimals=5)
test = test.drop_duplicates(keep='first')

test = test.pivot_table(values='values',
                        index=['STORETID', 'COUNT_STOR', 'MEAN_Latit', 'MEAN_Longi', 'satellite', 'tile', 'date_max', 'capture_date', 'date_min'],
                        columns=['band'],
                        aggfunc='mean').reset_index()


data_table.DataTable(test, num_rows_per_page=10)

# Load in *in situ* data

In [ ]:
uploaded2 = files.upload()
meas_sd = pd.read_csv(io.BytesIO(uploaded2['mi_secchi_V3.csv']))[["STORETID", "Date Sampled", "secchi_m"]]
meas_sd['Date Sampled'] = pd.to_datetime(meas_sd['Date Sampled'], format = '%Y-%m-%d')
data_table.DataTable(meas_sd, num_rows_per_page=10)

In [ ]:
test_merge = test.merge(meas_sd,on='STORETID',how='left')
data_table.DataTable(test_merge, num_rows_per_page=10)

In [ ]:
test_merge_filt = test_merge[(test_merge['Date Sampled'] > test_merge['date_min']) & (test_merge['Date Sampled'] < test_merge['date_max'])].reset_index()
test_merge_filt = test_merge_filt.drop(['index'], axis=1)
data_table.DataTable(test_merge_filt, num_rows_per_page=10)

In [ ]:
df = pd.DataFrame(test_merge_filt)
df2 = df.drop_duplicates(subset=['STORETID', 'Date Sampled'], keep=False)
data_table.DataTable(df2, num_rows_per_page=10)

In [ ]:
out_dir = os.path.expanduser('~/Downloads')
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

output_filepath = os.path.join(out_dir, 'sentinel_micorps_joined_V4.csv')
df2.to_csv(output_filepath)